We need to make the cooccurrence matrices to then calculate the pointwise mutual information (PMI) values to pass to the Dynamic Word Embedding model. 

In [ ]:
from csv import DictReader
from collections import Counter

# read in the freq counter
totalFreqDistFile = '/Users/bkitano/Desktop/Classes/spring_2019/thesis/embeddings/totalFreqDist.csv'

with open(totalFreqDistFile) as f:
    reader = DictReader(f, fieldnames=['word', 'count'])
    freqDist = Counter({row['word']: int(row['count']) for row in reader})

print(freqDist.most_common(10))
            

In [ ]:
print(sum(freqDist.values()))
# 463,231,888 words

In [ ]:
frequentWords = [w for w in freqDist.keys() if freqDist[w] > 200]
# 71670, 3x than paper

In [ ]:
print(len(frequentWords))
print(frequentWords[0:10])

In [ ]:
# create and save hashmap of word to ids
wordToID = dict(zip(frequentWords, range(len(freqDist))))

In [ ]:
IDToWord = dict([(wordToID[word], word) for word in wordToID.keys()])

In [ ]:
import numpy as np
from scipy.sparse import dok_matrix
from scipy.sparse import csr_matrix

def cooccurDummy( args, wordMap, V ):

    word = args[0] 
    window = args[1] 
    df = dok_matrix((V, V))
    for coword in window:
        try:
            wordID = wordMap[word]
            cowordID = wordMap[coword]
            df[wordID, cowordID] += 1.0
        except:
            print(word)
    csr = df.tocsr()
    return csr

def cooccur(args):
    return cooccurDummy(args, wordToID, len(frequentWords))

In [ ]:
import nltk
import os
from os import listdir
from os.path import isfile, join

textPath = '/Users/bkitano/Desktop/Classes/Spring_2019/thesis/corpus/cleaned_txt/'

textFiles = [f for f in listdir(textPath) if isfile(join(textPath, f))]
cooccurrencePath = '/Users/bkitano/Desktop/Classes/spring_2019/thesis/embeddings/cooccur/'
L = 5 # window size, or L in the paper
# window: x1 x2 ... xL W y1 y2 ... yL

# https://stackoverflow.com/questions/17458751/python-symmetric-word-matrix-using-nltk
# need to parallelize

with open(textPath + textFiles[0], 'r+') as f:
    text = f.read()
    tokens = [w for w in nltk.word_tokenize(text) if w in frequentWords]
#     for i in range(L, len(tokens)-1):

In [ ]:
wordTarget = tokens[L:len(tokens) - L]
windowTarget = [tokens[i-L:i + L + 1] for i in range(L, len(tokens) - L)]

pairs = list(zip(wordTarget, windowTarget))
#     for i in range(L, L+2):
#         word = tokens[i]
#         window = tokens[i-L : i + L + 1]
#         asdf((word, window, df, wordToID))

In [ ]:
h = cooccur(pairs[1])
print(h)

In [ ]:
from multiprocessing import Pool

with Pool(processes=4) as pool:
    results = pool.map(cooccur, pairs)
    pool.close()
    pool.join()
    print('All sub-processes done.')

In [ ]:
def sum_sparse(m):
    x = np.zeros(m[0].shape)
    for a in m:
        ri = np.repeat(np.arange(a.shape[0]),np.diff(a.indptr))
        x[ri,a.indices] += a.data
    return x

In [ ]:
agg = sum_sparse(results)

In [ ]:
import pickle
pickle.dump(agg, cooccurrencePath + "test.p")